In [1]:
import json
from selenium import webdriver
import os
import re
from bs4 import BeautifulSoup

# from selenium.webdriver.chrome.options import Options

# webdriver-manager
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# selenium tools
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webdriver import WebElement
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep
import json
from datetime import datetime

In [2]:
#selenium start
driver = webdriver.Chrome()

In [3]:
# Fazendo a requisição

post_video = "https://www.linkedin.com/posts/fernandogomesfg_python-gis-geospatial-ugcPost-7184308650679689216-ljBu?utm_source=share&utm_medium=member_desktop"
post_texto = "https://www.linkedin.com/posts/thiagolisboapro_99-das-pessoas-aqui-no-in-vivem-na-roda-activity-7186371514911383552-g_U9?utm_source=share&utm_medium=member_desktop"
post_imagem = "https://www.linkedin.com/posts/randypitcherii_snowflake-snowpark-where-have-you-been-activity-7186451080187195392-IXLk?utm_source=share&utm_medium=member_desktop"
post_mult_imagem = "https://www.linkedin.com/posts/rodrigobcerqueira_%C3%A9-poss%C3%ADvel-utilizar-dados-e-m%C3%A9todos-estat%C3%ADsticos-ugcPost-7181702612499447808-jwVv?utm_source=share&utm_medium=member_desktop"


url = """
https://www.linkedin.com/feed/update/urn:li:activity:7186083318310834177?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A7186083318310834177%29
"""



driver.get(url)

In [4]:
def verify_sign_in():
    xpath_close_modal = "//icon[contains(@class, 'contextual-sign-in-modal__modal-dismiss-icon')]"
    try:
        WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, xpath_close_modal))
        ).click()
        return True
    except TimeoutException:
        return False
    
verify_sign_in()


False

In [5]:
def verify_content_unavailable():
    xpath_content_unavailable = "//a[@data-tracking-control-name='public_post_content-unavailable-join']"
    try:
        WebDriverWait(driver, 1.5).until(
            EC.element_to_be_clickable((By.XPATH, xpath_content_unavailable))
        )
        return True
    except TimeoutException:
        return False
    
if verify_content_unavailable():
    print("Conteúdo disponível apenas para usuários, parando o script!")

In [6]:
scrap = {}
for i in range(9):
    driver.execute_script(f"window.scrollTo(0, {300 * i});")
    sleep(.5)

sleep(.5)

#data-tracking-control-name="public_post_content-unavailable-join"


article_element = driver.find_element(by=By.TAG_NAME, value="article")
soup_article = BeautifulSoup(article_element.get_attribute("outerHTML"), "html.parser")

#Original Poster
xpath_post_header = "//div[@data-test-id='main-feed-activity-card__entity-lockup']"
header = soup_article.find("div", attrs={"data-test-id": "main-feed-activity-card__entity-lockup"})
header_content = [item.strip() for item in header.text.split("\n") if item.strip()]

scrap["autor_name"] = header.find(attrs={"data-tracking-control-name":"public_post_feed-actor-name"}).text.strip()
scrap["autor_subtitle"] = header_content[1]
scrap["post_age"] = header_content[2]

scrap["autor_img_src"] = header.find("img").get("src")
# print(autor_subtitle)

print(scrap)

{'autor_name': 'Abraham Lincoln', 'autor_subtitle': 'UI Designer | UX Designer | Product Designer | Designer de Interface do Usuário | Designer de Experiência do Usuário | Figma', 'post_age': '6d', 'autor_img_src': 'https://static.licdn.com/aero-v1/sc/h/9c8pery4andzj6ohjkjp54ma2'}


In [7]:
scrap["content_text"] = soup_article.find("p", class_="attributed-text-segment-list__content").text
scrap["content_text"]

'Fala pessoal, tudo bem?\n\nEstou um tempinho sem postar, mas venho com novidades 🙂\n\nGostaria de compartilhar o projeto que participei na SouJunior\n\nÉ um local muito legal onde sempre tem algo para aprender, os mentores são super disponíveis e sempre ajudar os juninhos 🐣\n\nEsse projeto é sobre a dashboard que ajudei a desenvolver como ui/ux designer, não é a versão final, mas ajudou o time a ter uma visão do que a SouJunior precisava e fico feliz de saber que está em andamento.\n\nGostaria e agradecer ao Wouerner Brandão pela iniciativa da SouJunior, a Vanêssa Santana por sempre estar por perto para acompanhar o andamento, ao Vinicius L Miguel por sempre ajudar a todos e ser um excelente Head de UX e a todo o time da Ilimidados 😎\n\nMinha participação foi breve mas gostei bastante, para os juninhos que estão atrás de uma oportunidade no mercado profissional assim como eu, façam trabalho voluntário se puderem, vão ajudar a validar seus conhecimentos e aprender muita coisa nova.\n\n

In [8]:
scrap["content_imgs_src"] = []

try:
    article_element = driver.find_element(by=By.TAG_NAME, value="article")

    soup_article = BeautifulSoup(article_element.get_attribute("outerHTML"), "html.parser")
    content_imgs = soup_article.find(attrs={"data-test-id": "feed-images-content"})
    if content_imgs:
        content_imgs = content_imgs.find_all("img")
        print("Post contem imagens")
        scrap["content_imgs_src"] = [img.get("src") for img in content_imgs]

    content_video = soup_article.find_all("video")
    if content_video:
        print("Contem video")
        scrap["content_imgs_src"] = [video.get("data-poster-url") for video in content_video]


except NoSuchElementException:
    print("Tipo de postagem não suportado. Falha ao capturar conteúdo")

if len(scrap["content_imgs_src"]) == 0:
    print("Não foi encontrado imagem ou videos no post")
    
print(scrap["content_imgs_src"])

Post contem imagens
['https://media.licdn.com/dms/image/D4D22AQEj4DGfwhOg0g/feedshare-shrink_800/0/1713295773450?e=2147483647&v=beta&t=fy1PKyosV13-U0pE4Avta9eX4JSM7GhXqc2TM-h6XBY']


In [9]:
#reactions
reactions = soup_article.find("div", class_= "main-feed-activity-card__social-actions").text

scrap["reactions"] = [item.strip() for item in reactions.split("\n") if item.strip()]
print(scrap["reactions"])



['50', '9 Comments']


In [10]:
# comentários
scrap["comments"] = []
article_element = driver.find_element(by=By.TAG_NAME, value="article")
soup_article = BeautifulSoup(article_element.get_attribute("outerHTML"), "html.parser")


comments = soup_article.find_all("section", class_="comment")

for index, comment in enumerate(comments):
    current_comment = {}

    comment_header = comment.find(class_="comment__header").text.split("\n")
    comment_header_texts = [item.strip() for item in comment_header if item.strip()]
    current_comment["autor"] = comment_header_texts[0]
    current_comment["subtitle"] = comment_header_texts[1]
    current_comment["age"] = comment_header_texts[2]

    current_comment["profile_url"] = comment.find("a").get("href").split("?")[0]
    current_comment["profile_image_src"] = comment.find("img").get("src")
    current_comment["comment_text"] = comment.find(class_="comment__text").text
    current_comment["image_index"] = str(index)
    scrap["comments"].append(current_comment)

# post

In [11]:
def clear_text(text):
    # Remove caracteres não permitidos e converte para minúsculas
    text = re.sub(r'[^a-zA-Z0-9]', '_', text.lower())
    return text


today = datetime.now().strftime("%Y-%m-%d")
folder_name_autor = clear_text(scrap["autor_name"])
folder_name_post = clear_text(scrap["content_text"][:20])

export_path = f"scraped/{today}/{folder_name_autor}/{folder_name_post}"

if not os.path.exists(export_path):
    os.makedirs(export_path)

with open(f"{export_path}/data.json", "w", encoding="utf-8") as file:
    json.dump(scrap, file, ensure_ascii=False)

In [12]:
for comment in scrap["comments"]:
    print(comment)

{'autor': 'Abraham Lincoln', 'subtitle': 'UI Designer | UX Designer | Product Designer | Designer de Interface do Usuário | Designer de Experiência do Usuário | Figma', 'age': '6d', 'profile_url': 'https://br.linkedin.com/in/abrahamjunior', 'profile_image_src': 'https://static.licdn.com/aero-v1/sc/h/9c8pery4andzj6ohjkjp54ma2', 'comment_text': 'https://abrahamjunior.notion.site/Dashboard-Linkedin-c6c9fe5f92364828ab2a28a67288b76e?pvs=4', 'image_index': '0'}
{'autor': 'Vanêssa Santana', 'subtitle': 'Agile Coach | Gerente de Projetos | Head de Produtos | Scrum Trainer | Management 3.0 Facilitator', 'age': '5d', 'profile_url': 'https://br.linkedin.com/in/vanessa-patricia', 'profile_image_src': 'https://media.licdn.com/dms/image/D4D03AQHsnJBreBAQEw/profile-displayphoto-shrink_400_400/0/1679585890480?e=2147483647&v=beta&t=VptRyoJSzh3FjeKrGbQAhElHGndmO7-rEAutlJ9HpeM', 'comment_text': 'Abraham, foi muito bacana ter você no nosso time e desejo que a sua jornada na área de UX/UI seja um sucesso, 

In [13]:
driver.get(scrap["autor_img_src"])
sleep(.5)
driver.find_element(by=By.TAG_NAME, value="img").screenshot(f"{export_path}/autor_img.png")

for index, img in enumerate(scrap["content_imgs_src"]):
    sleep(.5)
    driver.get(img)
    driver.find_element(by=By.TAG_NAME, value="img").screenshot(f"{export_path}/content_img_{index}.png")

if scrap["comments"]:
    for index, comment in enumerate(scrap["comments"]):
        sleep(.5)
        driver.get(comment["profile_image_src"])

        image_output_path = f"{export_path}/comment_profile_photo_{index}.png"

        driver.find_element(by=By.TAG_NAME, value="img").screenshot(image_output_path)
        

In [14]:
driver.close()

In [15]:
import shutil
#copy to folder
input_folder = export_path
output_folder = "last_scrap"

#delete old folder
shutil.rmtree(output_folder, ignore_errors=True)

shutil.copytree(input_folder, output_folder)

'last_scrap'

In [16]:
data = {}

def get_autor(text):
    return {"autor_index": text, "type": "autor"}

data["autor"] = get_autor("1")


data



{'autor': {'autor_index': '1', 'type': 'autor'}}